In [113]:
##########
import pandas as pd
import numpy as np
import matplotlib as plt
import ast
import datetime
from ast import literal_eval
import sklearn
import json


In [ ]:
#####metadata preparation des donnees

In [114]:
metadata=pd.read_csv('movies_metadata.csv', delimiter = ',')


#1 supprimer les valeurs missing

metadata=metadata.dropna(subset=['id'])
metadata=metadata.dropna(subset=['title'])


#2 selectionner les film=released et transformer la variable adult en variable quali et supprimer les doublons id


#status released
metadata=metadata.loc[metadata['status']== 'Released']

#encode adult var
metadata1=pd.get_dummies(metadata, columns=["adult"])



#drop duplicates  

metadata2=metadata1.drop_duplicates()
metadata2=metadata2.drop_duplicates(subset='id', keep="first")


#selection variables


metadata2=metadata2[['genres', 
                     'id',
                     'original_language', 
                     'production_companies', 
                     'production_countries', 
                     'release_date',
                     'title'  , 
                     'vote_average',
                     'vote_count',
                     'adult_False', 
                     'adult_True'      ]]

/home/fitec/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [115]:
metadata2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44985 entries, 0 to 45465
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                44985 non-null  object 
 1   id                    44985 non-null  object 
 2   original_language     44975 non-null  object 
 3   production_companies  44985 non-null  object 
 4   production_countries  44985 non-null  object 
 5   release_date          44907 non-null  object 
 6   title                 44985 non-null  object 
 7   vote_average          44985 non-null  float64
 8   vote_count            44985 non-null  float64
 9   adult_False           44985 non-null  uint8  
 10  adult_True            44985 non-null  uint8  
dtypes: float64(2), object(7), uint8(2)
memory usage: 3.5+ MB


In [116]:
#####  listes des categories des variables genres, production_companies et production_countries


#metadata2['title'].value_counts(dropna=False)


#fonction pr obtenir une liste des categories (max 20 elements a priori) a partir des dictionnaires



def categorie (data, variable):

    data['genrestest'] = data[variable].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
    liste=data['genrestest'].str[0].value_counts()[:20].index.tolist()
    return liste


liste_genre=categorie(metadata2, 'genres')
liste_pcomp=categorie(metadata2, 'production_companies')
liste_pcount=categorie(metadata2, 'production_countries')

metadata2=metadata2.drop(['genrestest'], axis=1)





#########################fonction pour attribuer 1 a chaque element qui existe dans la liste des categories############################





def encoding_dic(data, variable, liste):
    
    #Colonne concernée
    #serie_col = data["genres"]
        
    #Liste des catégories conservées
    #liste = ['Drama', 'Comedy', 'Thriller', 'Romance', 'Action', 'Horror', 'Crime', 'Documentary', 'null']
    
    serie_col = data[variable]
    #Création de la colonne total : liste des catégories appartenant à la liste pour chaque ligne
    def add(x, liste_col):
        total = []
        if type(x) == str and x[0] == "[":
            a = ast.literal_eval(x)
            if len(a) > 0:
                for j in range(len(a)):
                    comp = a[j]["name"]
                    if comp in liste_col:
                        total.append(comp)
                if len(total) == 0:
                    total.append("null")
            else:
                total.append("null")
        return total
    
    total = serie_col.apply(lambda x : add(x, liste_col = liste))
    df = serie_col.to_frame()
    df["total"] = total
    
    #Création des colonnes pour le OneHotEncoding
    for genre in liste:
        df[genre] = 0
    
    #Complétion des colonnes OneHotEncoding grâce à la colonne total
    def add2(x,genre_cherche):
        for genre in x["total"]:
            if genre == genre_cherche:
                return 1
        return 0
    
    for genre in liste:
        df[genre] = df.apply(lambda x : add2(x, genre_cherche = genre), axis=1)
    
    return df



##########################################################""

dfgenres = encoding_dic(data=metadata2, variable="genres", liste=liste_genre)
dfgenres=dfgenres.drop(['genres', 'total'], axis=1)
dfprodcomp = encoding_dic(data=metadata2, variable="production_companies", liste=liste_pcomp)
dfprodcomp=dfprodcomp.drop(['production_companies', 'total'], axis=1)
dfprodcount = encoding_dic(data=metadata2, variable="production_countries", liste=liste_pcount)
dfprodcount=dfprodcount.drop(['production_countries', 'total'], axis=1)



#append ou concat moviedataset original avc les 3 dataframes dfgenres, dfprodcomp et  dfprodcount


datamovienew=metadata2.append(dfgenres, ignore_index=True)
datamovienew1=datamovienew.append(dfprodcomp,ignore_index=True )
datamovienew2=datamovienew1.append(dfprodcount,ignore_index=True )

datamovienew2=datamovienew2.drop(['genres', 'production_companies', 'production_countries'], axis=1)




#####keywords




keywords=pd.read_csv('keywords.csv', delimiter = ',')
keywords=keywords.dropna(subset=['id'])
keywords=keywords.drop_duplicates(subset='id', keep="first")

liste_key=categorie(keywords, 'keywords')

keywords=keywords.drop(['genrestest'], axis=1)
dfkey= encoding_dic(data=keywords, variable="keywords", liste=liste_key)
dfkey=dfkey.drop(['keywords', 'total'], axis=1)
datakey=keywords.append(dfkey, ignore_index=True)
datakey=datakey.drop(['keywords'], axis=1)

####merge entre keywords et datamovienew2   

#datakey['id']=datakey['id'].astype(str)  
##datakey['id'] = datakey['id'].str.replace('.0', '')
#datamovienew3=pd.merge(datamovienew2,datakey, on='id')




####FIN



In [123]:
datakey['id']

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
90859        NaN
90860        NaN
90861        NaN
90862        NaN
90863        NaN
Name: id, Length: 90864, dtype: float64

In [117]:
datamovienew2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179940 entries, 0 to 179939
Data columns (total 68 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   id                                      44985 non-null  object 
 1   original_language                       44975 non-null  object 
 2   release_date                            44907 non-null  object 
 3   title                                   44985 non-null  object 
 4   vote_average                            44985 non-null  float64
 5   vote_count                              44985 non-null  float64
 6   adult_False                             44985 non-null  float64
 7   adult_True                              44985 non-null  float64
 8   Drama                                   44985 non-null  float64
 9   Comedy                                  44985 non-null  float64
 10  Action                                  44985 non-null  

In [118]:
datamovienew2['id']

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
179935      NaN
179936      NaN
179937      NaN
179938      NaN
179939      NaN
Name: id, Length: 179940, dtype: object

In [119]:
datakey['id']

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
90859        NaN
90860        NaN
90861        NaN
90862        NaN
90863        NaN
Name: id, Length: 90864, dtype: float64

In [100]:
keywords=pd.read_csv('keywords.csv', delimiter = ',')
keywords=keywords.dropna(subset=['id'])
keywords=keywords.drop_duplicates(subset='id', keep="first")



In [101]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45432 non-null  int64 
 1   keywords  45432 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [102]:
liste_key=categorie(keywords, 'keywords')

keywords=keywords.drop(['genrestest'], axis=1)
dfkey= encoding_dic(data=keywords, variable="keywords", liste=liste_key)
dfkey=dfkey.drop(['keywords', 'total'], axis=1)

In [103]:
dfkey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 46418
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   woman director    45432 non-null  int64
 1   independent film  45432 non-null  int64
 2   based on novel    45432 non-null  int64
 3   musical           45432 non-null  int64
 4   female nudity     45432 non-null  int64
 5   sex               45432 non-null  int64
 6   prison            45432 non-null  int64
 7   suicide           45432 non-null  int64
 8   stand-up comedy   45432 non-null  int64
 9   paris             45432 non-null  int64
 10  new york          45432 non-null  int64
 11  biography         45432 non-null  int64
 12  sport             45432 non-null  int64
 13  gay               45432 non-null  int64
 14  london england    45432 non-null  int64
 15  martial arts      45432 non-null  int64
 16  holiday           45432 non-null  int64
 17  suspense          45432 non-nul

In [104]:






datakey=keywords.append(dfkey, ignore_index=True)
datakey=datakey.drop(['keywords'], axis=1)

In [105]:
datakey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90864 entries, 0 to 90863
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                45432 non-null  float64
 1   woman director    45432 non-null  float64
 2   independent film  45432 non-null  float64
 3   based on novel    45432 non-null  float64
 4   musical           45432 non-null  float64
 5   female nudity     45432 non-null  float64
 6   sex               45432 non-null  float64
 7   prison            45432 non-null  float64
 8   suicide           45432 non-null  float64
 9   stand-up comedy   45432 non-null  float64
 10  paris             45432 non-null  float64
 11  new york          45432 non-null  float64
 12  biography         45432 non-null  float64
 13  sport             45432 non-null  float64
 14  gay               45432 non-null  float64
 15  london england    45432 non-null  float64
 16  martial arts      45432 non-null  float6

In [111]:
datamovienew2['id']

0           862
1          8844
2         15602
3         31357
4         11862
          ...  
179935      NaN
179936      NaN
179937      NaN
179938      NaN
179939      NaN
Name: id, Length: 179940, dtype: object

In [106]:
datakey['id']

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
90859        NaN
90860        NaN
90861        NaN
90862        NaN
90863        NaN
Name: id, Length: 90864, dtype: float64

In [108]:
datakey['id']=datakey['id'].astype(str)

In [109]:
datakey['id']

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
90859        nan
90860        nan
90861        nan
90862        nan
90863        nan
Name: id, Length: 90864, dtype: object

In [110]:
#datakey['id'] = datakey['id'].str.replace('.0', '')


In [99]:
datakey['id']

0          862.0
1         8844.0
2        15602.0
3        31357.0
4        11862.0
          ...   
90859        NaN
90860        NaN
90861        NaN
90862        NaN
90863        NaN
Name: id, Length: 90864, dtype: float64

In [83]:
datakey.info() !!!!!!!!!!!!!!!!!!!!!!!!!!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90864 entries, 0 to 90863
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                90864 non-null  object 
 1   woman director    45432 non-null  float64
 2   independent film  45432 non-null  float64
 3   based on novel    45432 non-null  float64
 4   musical           45432 non-null  float64
 5   female nudity     45432 non-null  float64
 6   sex               45432 non-null  float64
 7   prison            45432 non-null  float64
 8   suicide           45432 non-null  float64
 9   stand-up comedy   45432 non-null  float64
 10  paris             45432 non-null  float64
 11  new york          45432 non-null  float64
 12  biography         45432 non-null  float64
 13  sport             45432 non-null  float64
 14  gay               45432 non-null  float64
 15  london england    45432 non-null  float64
 16  martial arts      45432 non-null  float6

In [93]:

datakey['id']

0          862 
1         8844 
2         15 2 
3        31357 
4        11862 
          ...  
90859       nan
90860       nan
90861       nan
90862       nan
90863       nan
Name: id, Length: 90864, dtype: object

In [94]:
tt=pd.merge(datamovienew2,datakey,on='id')


In [80]:
datakey["id"]= datakey["id"].astype(str)
     
datakey=datakey.dropna(subset=['id'])

datakey = datakey.apply (pd.to_numeric, errors='coerce')
datakey = datakey.dropna()

datakey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                0 non-null      float64
 1   woman director    0 non-null      float64
 2   independent film  0 non-null      float64
 3   based on novel    0 non-null      float64
 4   musical           0 non-null      float64
 5   female nudity     0 non-null      float64
 6   sex               0 non-null      float64
 7   prison            0 non-null      float64
 8   suicide           0 non-null      float64
 9   stand-up comedy   0 non-null      float64
 10  paris             0 non-null      float64
 11  new york          0 non-null      float64
 12  biography         0 non-null      float64
 13  sport             0 non-null      float64
 14  gay               0 non-null      float64
 15  london england    0 non-null      float64
 16  martial arts      0 non-null      float64
 17  holiday  

woman director                  1316
independent film                 700
based on novel                   441
musical                          386
female nudity                    371
sex                              307
prison                           283
suicide                          276
stand-up comedy                  274
paris                            259
new york                         233
biography                        231
sport                            224
gay                              217
london england                   213
martial arts                     213
holiday                          208
suspense                         183
male nudity                      178
japan                            168
world war ii                     162
rape                             146
monster                          144
love triangle                    135
father son relationship          134
spy                              134
detective                        127
b

In [47]:
datamovienew2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179940 entries, 0 to 179939
Data columns (total 71 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   genres                                  44985 non-null  object 
 1   id                                      44985 non-null  object 
 2   original_language                       44975 non-null  object 
 3   production_companies                    44985 non-null  object 
 4   production_countries                    44985 non-null  object 
 5   release_date                            44907 non-null  object 
 6   title                                   44985 non-null  object 
 7   vote_average                            44985 non-null  float64
 8   vote_count                              44985 non-null  float64
 9   adult_False                             44985 non-null  float64
 10  adult_True                              44985 non-null  